# Data Analysis Programming --- Assignment 3

                                                             Tiffany Sung (from Group 3)

### Set up

In [1]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### (Q1) Read in data
Read in both data and use '\t' as seperator.

In [2]:
gold = pd.read_csv('gold.txt',sep='\t',header=None,names=['url', 'category'])
labels = pd.read_csv('labels.txt',sep = '\t',header= None, names=['turk','url', 'category'])

#gold[:5]
#labels[:5]

### (Q2) Split into two DataFrames
* Step 1: Create a mask which the 'url' in labels is also in gold set. 
* Step 2: Use the mask to split the data frame into labels_on_gold and label_unknown.

In [3]:
on_gold = labels['url'].isin(gold['url'])
labels_on_gold = labels[on_gold].drop_duplicates()
labels_unknown = labels[~on_gold].drop_duplicates()

### (Q3) Compute accuracies of turks
* Step 1: By merging label_on_gold and gold, we get two 'category' columns from both data frame. Then use the mask to see if the category from both data frame are matched. The column 'Bingo' is where two categories are matched.
* Step 2: The average correctness of rating is calculated from the mean of 'Bingo'
* Step 3: By groupby('turk') and count the amount of category each turks rated, we get the number of ratings.

In [21]:
correct_ratings = labels_on_gold.merge(gold,
                                      left_on = 'url',
                                      right_on = 'url',
                                      suffixes=['_turk','_gold'])
correct = correct_ratings['category_turk'] == correct_ratings['category_gold']
correct_ratings['Bingo'] = correct
#correct_ratings[:10]

In [22]:
correctness = correct_ratings[['turk','Bingo']]
correctness = correctness.groupby('turk').mean()
#correctness[:5]

In [23]:
num_ratings = DataFrame(correct_ratings.groupby('turk')['category_turk'].count())
#num_ratings[:5]

In [7]:
rater_goodness = correctness.merge(num_ratings,left_on = 'turk',right_on = 'turk')
rater_goodness.columns = ['average correctness of ratings','number of ratings'] #change the column name
rater_goodness[:10]

,average correctness of ratings,number of ratings
turk,,
A112DVP1KG4QZU,1.000000,1
A1253FXHCZ9CWM,0.517241,29
A12CY1Q7XKJJDE,1.000000,1
A12RE8G66WTO8B,0.750000,20
A12Y1GTGIQDGRA,0.333333,3
A13CEW9JGDWGX1,1.000000,1
A13OE9GBRJ0S2U,0.750000,4
A14IQ4GLNWNPOJ,1.000000,1
A153PKAL7OAY36,0.722973,148


### (Q4) Odds ratios
* Step 1: Define a function that calculates the odds of success.
* Step 2: The column 'odds' is created by using the odds_ratios function to map through the 'average correctness of ratings'.

In [8]:
def odds_ratios(s):
    odds = s*1.0 / (1.001-s)
    return odds

rater_goodness['odds'] = rater_goodness['average correctness of ratings'].map(odds_ratios)
rater_goodness[:10]

,average correctness of ratings,number of ratings,odds
turk,,,
A112DVP1KG4QZU,1.000000,1,1000.000000
A1253FXHCZ9CWM,0.517241,29,1.069214
A12CY1Q7XKJJDE,1.000000,1,1000.000000
A12RE8G66WTO8B,0.750000,20,2.988048
A12Y1GTGIQDGRA,0.333333,3,0.499251
A13CEW9JGDWGX1,1.000000,1,1000.000000
A13OE9GBRJ0S2U,0.750000,4,2.988048
A14IQ4GLNWNPOJ,1.000000,1,1000.000000
A153PKAL7OAY36,0.722973,148,2.600369


### (Q5) Most accurate turks
* I use a mask to filter out the turks with at least 20 rates.
* Then, list out the mose accurate turks by sort function.

In [33]:
at_least_20 = rater_goodness['number of ratings'] >= 20
rater_goodness[at_least_20].sort_values(by='average correctness of ratings',ascending = False)[:10]

,average correctness of ratings,number of ratings,odds,number
turk,,,,
A2U0R4X38GUKZE,0.950000,20,18.627451,20
A22C0PJUBFJTI0,0.916667,36,10.869565,36
A23YQUBXZPKILZ,0.875000,24,6.944444,24
ATVALOQVDCMZW,0.854369,103,5.826657,103
A1HIXWH4OXT8S4,0.825000,40,4.687500,40
A3220HG1O83HQ4,0.818182,22,4.475385,22
A32W20KGQXS0LL,0.800000,25,3.980100,25
A20PWAB7G3HDHU,0.800000,20,3.980100,20
AJSJVK40F5HM6,0.785714,28,3.649635,28


### (Q6) Rating counts versus accuracy

* First, we perform a simple linear regression to see if there is a significant relationship.
* The regression results shows that, the number of rating counts has negative relationship with accuracy (odds). 
* This indicates that when turks rate more, they tend to become less accurate.

In [10]:
import statsmodels.api as sm
from patsy import dmatrices

In [11]:
rater_goodness['number'] = rater_goodness['number of ratings']
y, X=dmatrices('odds~number',data= rater_goodness,return_type='dataframe')
model=sm.OLS(y, X)
result=model.fit()
print result.summary()

                            OLS Regression Results                            
Dep. Variable:                   odds   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     26.77
Date:                Thu, 09 Aug 2018   Prob (F-statistic):           4.52e-07
Time:                        00:33:03   Log-Likelihood:                -2019.6
No. Observations:                 269   AIC:                             4043.
Df Residuals:                     267   BIC:                             4050.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    386.5694     30.441     12.699      0.0

### (Q7) Overall predicted odds

* Step 1:Create a list of the turks that rated more gold set urls than the other 75% of turks using the 'url_75' mask
* Step 2:Merge the rate record which is not in the gold set with the more than 75% turks of rater to get odds.
* Step 3:groupby 'url' and 'category' to calculate the product of odds.

In [34]:
url_75=rater_goodness['number of ratings']>(rater_goodness['number of ratings'].quantile(0.75))
url_75_turks=rater_goodness[url_75]

labels_unknown_odds=labels_unknown.merge(url_75_turks,left_on='turk',right_on='turk')
#labels_unknown_odds[:5]

In [13]:
overall_odds=DataFrame(labels_unknown_odds.groupby(['url','category'])['odds'].agg(prod))
overall_odds.columns = ['overall odds'] #change the column name
overall_odds[:10]

overall odds
url                                                category              
http://0-101.net                                   G             2.297602
http://000.cc                                      G             1.485456
http://0000.jp                                     G            14.339248
http://000relationships.com                        G             3.841962
                                                   P             1.851852
http://000vitamins.com                             G             4.033641
http://000webhost.com                              G            11.892639
http://003my.com                                   G             4.861524
http://007absolutehardcore.com/early_cumshots/i... P             3.346522
                                                   R            12.290450

### (Q8) Predicted categories
* Step 1: First, unstack the overall_odds by category.
* Step 2: Using the idxmax() to get the index of max value of overall odds.
* Step 3: Using the max() to get max value of overall odds.
* Step 4: Merge step2 and step3 together using 'url' as index and 'top category','top odds' as columns

In [37]:
category_unstack = overall_odds.unstack('category')

top_category = DataFrame(category_unstack['overall odds'].T.idxmax())
#top_category[:10]

In [38]:
top_odds = DataFrame(category_unstack['overall odds'].T.max())
#top_odds[:10]

In [39]:
result_75 = top_category.merge(top_odds,left_on = 'url',right_on = 'url')
result_75.columns=['top category','top odds'] #rename the columns
result_75[:10]

,top category,top odds
url,,
http://0-101.net,G,2.297602
http://000.cc,G,1.485456
http://0000.jp,G,14.339248
http://000relationships.com,G,3.841962
http://000vitamins.com,G,4.033641
http://000webhost.com,G,11.892639
http://003my.com,G,4.861524
http://007absolutehardcore.com/early_cumshots/index.html,X,293.252871
http://007swz.cn,G,1.485456


### (Q9) Predicted categories using more turks

* Step 1: Repeat Q7 and Q8, but change the quantile 75% to 25% to get result_25.
* Step 2: Merge result_75 and result_25.
* Step 3: Use crosstab to find out where the most errors are.

In [18]:
#repeat Q7 but with 25% confident level
url_25=rater_goodness['number of ratings']>rater_goodness['number of ratings'].quantile(0.25)
url_25_turks=rater_goodness[url_25]
labels_unknown_odds_25=labels_unknown.merge(url_25_turks,left_on='turk',right_on='turk')

overall_odds_25=DataFrame(labels_unknown_odds_25.groupby(['url','category'])['odds'].agg(prod))
overall_odds_25.columns = ['overall odds']

In [19]:
#repeat Q8 with 25%
category_unstack_25 = overall_odds_25.unstack('category')
top_category_25 = DataFrame(category_unstack_25['overall odds'].T.idxmax())
top_odds_25 = DataFrame(category_unstack_25['overall odds'].T.max())
result_25 = top_category_25.merge(top_odds_25,left_on = 'url',right_on = 'url')
result_25.columns=['top category','top odds']

In [20]:
merge_75_25 = result_75.merge(result_25,
                              left_on = 'url',
                              right_on = 'url',
                              suffixes = ['_75','_25'])

most_errors = pd.crosstab(merge_75_25['top category_75'],merge_75_25['top category_25'])
most_errors

top category_25,G,P,R,X
top category_75,,,,
G,8280,586,180,214
P,196,352,52,25
R,22,34,127,25
X,28,5,28,456
